# Crewai + Personalised Educational Assistant

#### Refered documentation: 
#### https://docs.crewai.com/en/guides/crews/first-crew
#### https://docs.crewai.com/en/learn/create-custom-tools
#### https://docs.crewai.com/en/tools/search-research/serperdevtool
#### https://serper.dev/

In [15]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool
from pydantic import BaseModel
from typing import List
import os
from openai import OpenAI
import json
from crewai.tools import tool

In [17]:
os.environ["SERPER_API_KEY"] = "<API KEY>"
os.environ["OPENAI_API_KEY"] = "<API KEY>"

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")

In [19]:
class LearningMaterialsModel(BaseModel):
    articles: List[str]
    exercises: List[str]


class QuizQuestion(BaseModel):
    question: str
    options: List[str]
    answer: str


class QuizModel(BaseModel):
    questions: List[QuizQuestion]


class ProjectIdeasModel(BaseModel):
    ideas: List[str]

In [54]:
client = OpenAI()

@tool("project_suggestion_tool")
def project_suggestion_tool(topic: str, expertise_level: str) -> str:
    """
    Uses OpenAI LLM to generate realistic project ideas for the given topic
    and expertise level. Returns a JSON string containing the ideas list.
    """
    prompt = f"""
    Generate 4 practical, high-quality project ideas for a learner studying the topic:
    "{topic}"

    Their expertise level is: {expertise_level}.

    Requirements:
    - The project ideas MUST be realistic, implementable, and topic-specific.
    - Avoid generic suggestions.
    - Output ONLY a JSON list of strings, example:
      ["idea1", "idea2", "idea3", "idea4"]
    """

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
    )

    ideas_raw = response.choices[0].message.content.strip()

    try:
        ideas = json.loads(ideas_raw)
    except:
        ideas = [ideas_raw]

    return json.dumps({"ideas": ideas})

In [56]:
serper_tool = SerperDevTool()

In [58]:
learning_material_agent = Agent(
    role="Learning Material Agent",
    goal="Curate article-based learning materials and generate exercises for user topics.",
    backstory=(
        "An expert educational content curator trained in extracting high-quality articles "
        "from the web and generating meaningful practice exercises."
    ),
    tools=[serper_tool],
    allow_delegation=False,
    verbose=True,
)

quiz_agent = Agent(
    role="Quiz Creator Agent",
    goal="Generate personalised quizzes to evaluate user understanding.",
    backstory=(
        "A specialized assessment designer with experience in pedagogy and question formulation."
    ),
    tools=[],
    allow_delegation=False,
    verbose=True,
)

project_agent = Agent(
    role="Project Idea Agent",
    goal="Recommend project ideas tailored to the user's expertise level and topic.",
    backstory=(
        "A mentor-like agent that helps learners practice real-world skills through hands-on projects."
    ),
    tools=[project_suggestion_tool],
    allow_delegation=False,
    verbose=True,
)

In [60]:
materials_task = Task(
    description=(
        "Generate structured learning materials for the topic: {topic}. "
        "Use SERPER to extract high-quality ARTICLES only. "
        "Then generate EXERCISES related to the topic. "
        "Return output in LearningMaterialsModel JSON format."
    ),
    agent=learning_material_agent,
    expected_output="A JSON matching LearningMaterialsModel: {articles: [...], exercises: [...]}",
    output_json=LearningMaterialsModel,
)


quiz_task = Task(
    description=(
        "Create a personalized quiz for topic: {topic}. "
        "Generate 5–10 questions including multiple-choice and conceptual questions. "
        "Return output following QuizModel schema."
    ),
    agent=quiz_agent,
    expected_output="A JSON matching QuizModel: {questions: [...]}.",
    output_json=QuizModel,
)


project_task = Task(
    description=(
        "Suggest practical project ideas for the topic: {topic} "
        "and expertise level: {expertise_level}. "
        "Use the project_suggestion_tool to generate ideas. "
        "Return output following ProjectIdeasModel."
    ),
    agent=project_agent,
    expected_output="A JSON matching ProjectIdeasModel: {ideas: [...]}.",
    output_json=ProjectIdeasModel,
)

In [62]:
learning_assistant_crew = Crew(
    agents=[learning_material_agent, quiz_agent, project_agent],
    tasks=[materials_task, quiz_task, project_task],
    process=Process.sequential,
    verbose=True,
)

In [64]:
def run_learning_assistant(topic: str, expertise_level: str):
    inputs = {
        "topic": topic,
        "expertise_level": expertise_level,
    }

    result = learning_assistant_crew.kickoff(inputs=inputs)
    return result

### The customization/personalization comes here where the topic and difficulty level can be choosen here - it can also be obtained as an input from the user easily

In [66]:
result = run_learning_assistant("Sacoglossan sea slugs", "intermediate")
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 3b50b448-1bbd-42a2-9b4e-30a1197e9a0c                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Learning Material Agent                                                                                 │
│                                                                                                                 │
│  Task: Generate structured learning materials for the topic: Sacoglossan sea slugs. Use SERPER to extract       │
│  high-quality ARTICLES only. Then generate EXERCISES related to the topic. Return output in                     │
│  LearningMaterialsModel JSON format.                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Learning Material Agent                                                                                 │
│                                                                                                                 │
│  Thought: Thought: I need to find high-quality articles specifically about Sacoglossan sea slugs. These         │
│  articles should ideally provide detailed scientific or educational information about this group of sea slugs.  │
│  Then, based on that information, I will generate related exercises for learning.                               │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Project Idea Agent                                                                                      │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {"ideas": ["Conduct a comparative study on the feeding habits of different Sacoglossan species and their       │
│  impact on algal populations in a specific marine environment.", "Create a detailed field guide that documents  │
│  the various Sacoglossan species found in a local coastal area, including their morphology, habitat             │
│  preferences, and behaviors.", "Investigate the chemical ecology of Sacoglossan sea slugs by analyzing the      │
│  secondary metabolites they acquire from their algal diets and their potential ecological functions.", "Design  │
│  an aquarium-based experiment to test the effects of environmental variables (such as light intensity and       │
│  water temperature) on the feeding rates and reproductive success of a selected Sacoglossan species."]}         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

{'ideas': ['Conduct a comparative study on the feeding habits of different Sacoglossan species and their impact on algal populations in a specific marine environment.', 'Create a detailed field guide that documents the various Sacoglossan species found in a local coastal area, including their morphology, habitat preferences, and behaviors.', 'Investigate the chemical ecology of Sacoglossan sea slugs by analyzing the secondary metabolites they acquire from their algal diets and their potential ecological functions.', 'Design an aquarium-based experiment to test the effects of environmental variables (such as light intensity and water temperature) on the feeding rates and reproductive success of a selected Sacoglossan species.']}


╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 7af2efda-3c93-4246-b6eb-18df191a9d54                                                                     │
│  Agent: Project Idea Agent                                                                                      │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 3b50b448-1bbd-42a2-9b4e-30a1197e9a0c                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: {"ideas": ["Conduct a comparative study on the feeding habits of different Sacoglossan species   │
│  and their impact on algal populations in a specific marine environment.", "Create a detailed field guide that  │
│  documents the various Sacoglossan species found in a local coastal area, including their morphology, habitat   │
│  preferences, and behaviors.", "Investigate the chemical ecology of Sacoglossan sea slugs by analyzing the      │
│  secondary metabolites they acquire from their algal diets and their potential ecological functions.", "Design  │
│  an aquarium-based experiment to test the effects of environmental variables (such as light intensity and       │
│  water temperature) on the feeding rates and reproductive success of a selected Sacoglossan species."]}         │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [68]:
def print_learning_assistant_output(result):
    print("\n" + "="*80)
    print("📘 LEARNING MATERIALS")
    print("="*80)

    # ---- Learning Materials ----
    learning_data = result.tasks_output[0].json_dict
    articles = learning_data.get("articles", [])
    exercises = learning_data.get("exercises", [])

    print("\n🔹 Articles:")
    for i, art in enumerate(articles, 1):
        print(f"\n  {i}. {art}")

    print("\n🔹 Exercises:")
    for i, ex in enumerate(exercises, 1):
        print(f"\n  {i}. {ex}")

    print("\n" + "="*80)
    print("📝 QUIZ QUESTIONS")
    print("="*80)

    # ---- Quiz Questions ----
    quiz_data = result.tasks_output[1].json_dict
    questions = quiz_data.get("questions", [])

    for i, q in enumerate(questions, 1):
        print(f"\nQ{i}. {q['question']}")
        for opt_i, opt in enumerate(q['options'], 1):
            print(f"   {opt_i}) {opt}")
        print(f"   ✔ Answer: {q['answer']}")

    print("\n" + "="*80)
    print("💡 PROJECT IDEAS")
    print("="*80)

    # ---- Project Ideas ----
    project_data = result.tasks_output[2].json_dict
    ideas = project_data.get("ideas", [])

    for i, idea in enumerate(ideas, 1):
        print(f"\n  {i}. {idea}")

    print("\n" + "="*80)


In [70]:
print_learning_assistant_output(result)


📘 LEARNING MATERIALS

🔹 Articles:

  1. Sacoglossan sea slugs are a unique group of marine gastropods known for their remarkable ability to incorporate chloroplasts from the algae they consume into their own cells, a phenomenon known as kleptoplasty. These stolen chloroplasts, referred to as kleptoplasts, remain functional within the slugs' tissues, allowing them to perform photosynthesis for extended periods, sometimes lasting several months. This makes sacoglossan sea slugs among the few animals capable of utilizing photosynthesis to supplement their energy needs.

  2. The process of kleptoplasty involves the sacoglossan sea slugs using their specialized radular teeth to pierce algae cells and suck out the chloroplasts. Unlike other animals which digest chloroplasts entirely, sacoglossans maintain these plastids in a functional state within their digestive gland cells. Current research indicates that these slugs have evolved mechanisms to protect and sustain the chloroplasts, inclu